In [7]:
import sys

import pandas as pd

sys.path.append('/Users/samrelins/Documents/LIDA/dental_project/src/')

from baseline_data_prep import *
from education_data_prep import *
from naive_model_helpers import *
from platoon_data_prep import *
from tableone import TableOne
from sklearn.metrics import roc_auc_score, roc_curve

In [8]:
bib_dir = "/users/samrelins/documents/lida/dental_project/data/bib_data"
epi_path = os.path.join(bib_dir, "dental/dsurvey_5yo/data.csv")
epi_data = pd.read_csv(epi_path)
epi_data["caries_experience"] = (epi_data.ddsdmft > 0).astype("int")
epi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   entity_id          354 non-null    object 
 1   has_dsurvey_5yo    354 non-null    float64
 2   AgeMths            354 non-null    float64
 3   ddsdt              354 non-null    float64
 4   ddsmt              354 non-null    float64
 5   ddsft              354 non-null    float64
 6   ddsdmft            354 non-null    float64
 7   ddscareindex       354 non-null    float64
 8   caries_experience  354 non-null    int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 25.0+ KB


/opt/anaconda3/envs/DentalEnv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
edrecs_data = return_merged_edrecs_df(bib_dir, include_schools=True)

edrecs_caries = (epi_data[["entity_id", "caries_experience"]]
                 .merge(edrecs_data,
                        on="entity_id",
                        how="left"))

/opt/anaconda3/envs/DentalEnv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
platoon_data = return_platoon_decay_data(bib_dir)

edrecs_platoon = (platoon_data[["entity_id", "caries_experience"]]
                  .merge(edrecs_data,
                         on="entity_id",
                         how="left"))

/opt/anaconda3/envs/DentalEnv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
pd.set_option("display.max_rows", None)
epi_schools = pd.DataFrame(
    edrecs_caries.phonics_testestablishment1.value_counts()
).rename({"phonics_testestablishment1": "epi_count"}, axis=1)

platoon_schools = pd.DataFrame(
    edrecs_platoon.phonics_testestablishment1.value_counts()
).rename({"phonics_testestablishment1": "platoon_count"}, axis=1)

school_counts = (epi_schools.join(platoon_schools, how="left")
                 .fillna(0)
                 .sort_values("platoon_count", ascending=False))
school_counts

/opt/anaconda3/envs/DentalEnv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,epi_count,platoon_count
754480DC77A3,8,55.0
56E006DA6C39,13,36.0
80509FC2E343,12,33.0
1ED87E47C4AA,13,29.0
E375285975A9,7,25.0
B706691711C5,6,20.0
E8785173C21D,1,13.0
681DC294C2F3,8,13.0
254B4488A06F,5,8.0
F45CDBD9EBA7,6,7.0
